In [1]:
import os
import pymol
import subprocess
#import pyrosetta
#pyrosetta.init("-mute all")
#pyrosetta.init("-ignore_zero_occupancy false -mute all")

In [6]:
#Passo 0 - Separar as poses do ligante (opcional)
#entrada = [pdb for pdb in os.listdir("ache2/") if pdb[0:3] == "out"]
#for e in entrada:
    #nome = "PDBQT/" + e[3:-11] #Salva na pasta PDBQT. Ela já precisa existir
    #os.system("vina_split --input ache1/"+e+" --ligand "+nome) #considera que o pdbqt está na pasta ache1

In [2]:
#Passo 1 - converter tudo para .pdb
entrada = [pdb for pdb in os.listdir("ache3/") if pdb[0:3] == "out"]
if not os.path.isdir("PDB"):
    os.mkdir("PDB")
for e in entrada:
    pymol.cmd.load("ache3/" + e) #Considera que o pdbqt de entrada está na pasta ache1
    pymol.cmd.save("PDB/" + e[3:-11] + ".pdb", state = 0)
    pymol.cmd.reinitialize()

 PyMOL not running, entering library mode (experimental)


In [3]:
#Passo 2 - Cálculo dos scores com o Convex-PL
if not os.path.isdir("Convex_logs"):
    os.mkdir("Convex_logs")
for e in entrada:
    command = "./Convex-PL --receptor ache3.pdb --ligand PDB/" + e[3:-11]+".pdb >Convex_logs/"+e[3:-11] +".txt"
    os.system(command)

In [5]:
#Passo 3 - Ler e unificar os scores nos outputs  
arquivos = os.listdir("Convex_logs/")
sumario = open("sumario-ache3.csv", "w")
sumario.write("Ligante,score0,score1,score2,score3,score4,score5,score6,score7,score8,score9\n")
for a in arquivos:
    sumario.write(a[:-4])
    for linha in open("Convex_logs/" + a):
        l = linha.split()
        if l[0]=="model":
            sumario.write("," + l[-1])
    sumario.write("\n")
sumario.close()

In [ ]:
#Passo 4 - Reordem


In [31]:
#Passo 4 - Criar complexo para uso no Pyrosetta
pymol.cmd.load("PDB/Bendiocarb01.pdb")
pymol.cmd.load("ache1.pdb")
pymol.cmd.save("Complexos/comp(ache1_Bendiocarb01).pdb")
pymol.cmd.reinitialize()

In [2]:
#Passo 4 - Calcular scores com pyrosetta
scorefxn = pyrosetta.create_score_function("ref2015")
receptor = pyrosetta.pose_from_pdb("ache1.pdb")
pose = pyrosetta.pose_from_pdb("Complexos/comp(ache1_Bendiocarb01).pdb")

core.scoring.etable: {0} Starting energy table calculation
core.scoring.etable: {0} smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: {0} smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: {0} smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: {0} Finished calculating energy tables.
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBEval.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/DonStrength.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/AccStrength.csv
core.chemical.GlobalResidueTypeSet: {0} Finished initializing fa_standard residue type set.

RuntimeError: 

File: /home/benchmark/rosetta/source/src/core/io/pose_from_sfr/PoseFromSFRBuilder.cc:1442
[ ERROR ] UtilityExitException
ERROR: Unrecognized residue: <0>



In [24]:
import pandas as pd
s = pd.read_csv("sumario.csv")
s

,Ligante,score0,score1,score2,score3,score4,score5,score6,score7,score8,score9
0,Alpha-Naphthylacetate,26.133909,24.621640,22.756163,28.436816,25.740118,24.006415,25.255481,28.135910,22.493956,23.571553
1,Bendiocarb,29.877957,29.217636,30.147770,31.580845,27.755452,28.361817,26.944693,26.456617,29.739028,23.486266
2,Benzene,13.240412,13.259894,13.243959,13.252498,13.176008,18.313267,18.309578,18.360797,18.234834,18.171104
3,Galantamine,47.050271,41.804552,44.088906,39.625021,42.702093,39.402308,39.021841,43.586722,39.333840,36.171341
4,Glucose,19.330485,19.707008,22.135680,24.154523,16.303903,21.304034,22.956192,20.014570,17.718535,16.611958
...,...,...,...,...,...,...,...,...,...,...,...
174,ZINC968246,31.100666,29.045723,26.712144,26.904459,26.355205,27.094305,27.119460,26.439340,25.635414,25.470116
175,ZINC968250,30.376179,30.260811,30.404622,30.387821,26.758727,26.760107,26.721561,26.801511,26.725827,22.761201
176,ZINC968287,29.972781,25.849026,25.825254,31.222226,31.880683,31.843790,31.331961,29.200666,27.951185,27.582885
177,ZINC968468,32.627173,37.267640,29.378245,35.737565,33.793080,34.277310,33.145514,31.820119,36.064165,30.331835


In [19]:
len(arquivos)

179